In [ ]:
import scipy.integrate as spi
import numpy as np
import matplotlib.pylab as plt


class Meta:
    def __init__(self, n, c):
        # define parameters
        self.n_pop = n
        self.beta = 1.0*np.ones(n)
        self.gamma = 0.1*np.ones(n)
        self.nu = 0.0001*np.ones(n)
        self.mu = 0.0001*np.ones(n)
        self.m = c*np.ones((n,n)) 
        self.mig_rate = self.m - np.diag(np.diag(self.m))

        # determine initial conditions
        self.X0 = 0.1*np.ones(n)
        self.Y0=0.0*np.ones(n)
        self.Y0[0]=0.0001

        # define time range
        n_days = 7 * 365
        self.t_range = np.arange(0, n_days, 1)
    
    
    def diff_eqs(self, inp, t):
        """
        Main set of equations in the SIR model with metapopulations. 
        """
        n = self.n_pop
        Y = np.zeros((2*n))
        V = inp

        # iterate over amount of subpopulations
        for i in range(n):
            Y[i] = self.nu[i] - self.beta[i]*V[i]*V[n+i] - self.mu[i]*V[i]; 
            Y[n+i] = self.beta[i]*V[i]*V[n+i] - self.mu[i]*V[n+i] - self.gamma[i]*V[n+i]

            for j in range(n):
                Y[i] += self.mig_rate[i][j]*V[j] - self.mig_rate[j][i]*V[i];
                Y[n+i] += self.mig_rate[i][j]*V[n+j] - self.mig_rate[j][i]*V[n+i];
        return Y

# create Meta object with 2 subpopulations
values = [0.0, 0.0001, 0.1]
for value in values:
    meta = Meta(2,value)
    solution = spi.odeint(meta.diff_eqs, np.hstack((meta.X0,coupling.Y0)), meta.t_range)

    # plot results
    n = coupling.n_pop
    plt.plot(no_coupling.t_range/365.0, solution[:,0], color='blue')
    plt.plot(no_coupling.t_range/365.0, solution[:,1], color='red')
    plt.xlabel('Time (Years)')
    plt.ylabel('Susceptibles')
plt.show()

#     plt.plot(no_coupling.t_range/365.0, solution[:,0+n], color='blue')
#     plt.plot(no_coupling.t_range/365.0, solution[:,1+n], color='red')
#     plt.ylabel('Infected')
#     plt.xlabel('Time (Years)')
#     plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import random
import pandas as pd

class Gillespie:

    def __init__(self, X, Y, beta, gamma, mu):
        self.beta = beta
        self.gamma = gamma
        self.mu = mu
        self.delta = 0.01
        self.epsilon = 0.001
        self.N = 100
        self.X = X
        self.Y = Y
        self.Z = 0
        self.T = 0


    def get_rates(self, X, Y, Z):
        """
        Determine at which rate an event occurs.
        """
        rate_E1 = self.mu * self.N
        rate_E2 = self.beta * self.X * self.Y / self.N
        rate_E3 = self.gamma * self.Y
        rate_E4 = self.mu * self.X
        rate_E5 = self.mu * self.Y
        rate_E6 = self.mu * self.Z
        rate_E7 = self.delta   * 0
        rate_E8 = self.epsilon * self.X  * 0

        events = [rate_E1, rate_E2, rate_E3, rate_E4, rate_E5, rate_E6, rate_E7, rate_E8]

        return events

    def gillespie(self):
        """
        Choose the next time and event.
        """

        # Generate two random numbers between 0 and 1.
        random_time = np.random.rand()

        events = self.get_rates(self.X, self.Y, self.Z)

        Rtotal = sum(events)

        # Get reaction time
        tau = (1.0/Rtotal) * np.log((1.0/random_time))
        self.T += tau

        # Determine which event occurs
        # Make list of reaction intervals
        rates = []
        for i in events:
            rates.append(i/Rtotal)

        random_event = random.uniform(0, max(rates))

        # Find event to be executed based on closest number to
        # a random number in rates-array
        found = self.find_nearest(rates, random_event)

        # Retrieve which event is related to this interval
        for k in range(len(rates)):
            if rates[k] == found:
                index = k
                break

        # Birth
        if index == 0:
            self.X += 1

        # Transmission
        elif index == 1:
            self.Y += 1
            self.X -= 1

        # Recovery
        elif index == 2:
            self.Z += 1
            self.Y -= 1

        # Death
        elif index == 3:
            self.X -= 1
        elif index == 4:
            self.Y -= 1
        elif index == 5:
            self.Z -= 1
        
        # Import via immigration
        elif index == 6:
            self.Y += 1
            
        # Import via external infection
        else:
            self.Y += 1
            self.X -= 1
        
        return self.X, self.Y, self.Z, self.T

    def find_nearest(self, array, value):
        """
        Find nearest value in array.
        """

        array = np.asarray(array)
        idx = (np.abs(array - value)).argmin()
        return array[idx]


if __name__ == '__main__':

    # make n amount of subpopulations
    pop_1 = Gillespie(99, 1, 1/3, 0.01, 0.003)
    pop_2 = Gillespie(99, 1, 1/3, 0.01, 0.003)
    
    # TODO: choose random subpopulation to perform gillespie() on
    # TODO: for new state of this population, compute state of other populations on this time point
    
